In [7]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options  
from webdriver_manager.chrome import ChromeDriverManager  
from selenium.webdriver.common.by import By


from bs4 import BeautifulSoup
import time

In [10]:
chrome_options = Options()
chrome_options.add_argument("--headless") 
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=chrome_options)

In [11]:
url = 'https://ge.globo.com/futebol/brasileirao-serie-a/'
driver.get(url)

html = driver.page_source


driver.quit()

In [15]:
soup = BeautifulSoup(html, 'html.parser')


In [ ]:
sections = soup.find_all('section', class_="tabela tabela__pontos-corridos")
for i, section in enumerate(sections):
    print(f"Seção {i+1}:")
    print(section.get_text(strip=True))  
    print("-" * 40)  # 

In [ ]:
sections = soup.find_all('section',class_="tabela tabela__pontos-corridos")
for i, section in enumerate(sections):
    tables = section.find_all('table', class_="tabela__equipes tabela__equipes--com-borda")  
    if tables:
        print(f"Seção {i+1} contém {len(tables)} tabela(s):")
        for j, table in enumerate(tables):
            print(f"Tabela {j+1}:")
            rows = table.find_all('tr')  

            for row in rows:
                cols = row.find_all(['td', 'th'])  
                cols_text = [col.get_text(strip=True) for col in cols]  
                print(cols_text)  
            print("-" * 40)  

In [38]:
sections = soup.find_all('section', class_="tabela tabela__pontos-corridos")
for i, section in enumerate(sections):
    tables = section.find_all('table',class_="tabela__pontos")  
    if tables:
        print(f"Seção {i+1} contém {len(tables)} tabela(s):")
        for j, table in enumerate(tables):
            print(f"Tabela {j+1}:")
            rows = table.find_all('tr')  

            for row in rows:
                cols = row.find_all(['td', 'th'])  
                cols_text = [col.get_text(strip=True) for col in cols]  
                print(cols_text)  
            print("-" * 40)  

Seção 1 contém 1 tabela(s):
Tabela 1:
['P', 'J', 'V', 'E', 'D', 'GP', 'GC', 'SG', '%', 'ÚLT. JOGOS']
['60', '29', '18', '6', '5', '47', '25', '22', '68', '']
['57', '29', '17', '6', '6', '46', '20', '26', '65', '']
['55', '29', '16', '7', '6', '38', '29', '9', '63', '']
['51', '28', '15', '6', '7', '45', '32', '13', '60', '']
['47', '29', '14', '5', '10', '38', '32', '6', '54', '']
['46', '28', '12', '10', '6', '37', '26', '11', '54', '']
['45', '29', '13', '6', '10', '39', '33', '6', '51', '']
['43', '29', '12', '7', '10', '35', '29', '6', '49', '']
['40', '28', '10', '10', '8', '40', '41', '-1', '47', '']
['37', '28', '10', '7', '11', '32', '38', '-6', '44', '']
['35', '29', '10', '5', '14', '33', '37', '-4', '40', '']
['35', '29', '9', '8', '12', '36', '42', '-6', '40', '']
['34', '29', '8', '10', '11', '34', '38', '-4', '39', '']
['34', '29', '8', '10', '11', '33', '39', '-6', '39', '']
['31', '27', '8', '7', '12', '27', '31', '-4', '38', '']
['30', '28', '8', '6', '14', '22', '30'

In [39]:
import re
from tabulate import tabulate

sections = soup.find_all('section', class_="tabela tabela__pontos-corridos")

tabela_completa = []

cabecalho = ['Posição', 'Time', 'P', 'J', 'V', 'E', 'D', 'GP', 'GC', 'SG', '%', 'ÚLT. JOGOS']
tabela_completa.append(cabecalho)

for section in sections:
    tabela_equipes = section.find_all('table', class_="tabela__equipes tabela__equipes--com-borda")
    
    tabela_pontos = section.find_all('table', class_="tabela__pontos")
    
    if tabela_equipes and tabela_pontos:  
        for table_equipes, table_pontos in zip(tabela_equipes, tabela_pontos):
            rows_equipes = table_equipes.find_all('tr')
            rows_pontos = table_pontos.find_all('tr')
            
            for row_equipe, row_ponto in zip(rows_equipes, rows_pontos):
                cols_equipe = row_equipe.find_all(['td', 'th'])
                cols_text_equipe = [col.get_text(strip=True) for col in cols_equipe]

                if len(cols_text_equipe) > 1:
                    nome_time = cols_text_equipe[1].replace('BOT', '').replace('PAL', '').replace('FOR', '').replace('FLA', '').replace('SAO', '').replace('INT', '').replace('BAH', '').replace('CRU', '').replace('CAM', '').replace('VAS', '').replace('GRE', '').replace('CRI', '').replace('RBB', '').replace('JUV', '').replace('CAP', '').replace('FLU', '').replace('VIT', '').replace('COR', '').replace('CUI', '').replace('ACG', '').strip()
                    cols_text_equipe[1] = nome_time

                cols_ponto = row_ponto.find_all(['td', 'th'])
                cols_text_ponto = [col.get_text(strip=True) for col in cols_ponto]

                ultimos_jogos_col = row_ponto.find_all('td', class_="classificacao__pontos classificacao__pontos--ultimos_jogos")
                ultimos_jogos_resultados = []
                
                for col in ultimos_jogos_col:
                    spans = col.find_all('span')
                    for span in spans:
                        span_class = span.get('class', [])
                        
                        if "classificacao__ultimos_jogos--v" in span_class:
                            ultimos_jogos_resultados.append('V')  # Vitória
                        elif "classificacao__ultimos_jogos--e" in span_class:
                            ultimos_jogos_resultados.append('E')  # Empate
                        elif "classificacao__ultimos_jogos--d" in span_class:
                            ultimos_jogos_resultados.append('D')  # Derrota

                linha_completa = cols_text_equipe + cols_text_ponto
                
                
                if linha_completa[-1] == '':
                    linha_completa.pop()  
               
                if ultimos_jogos_resultados:  
                    linha_completa.append(' '.join(ultimos_jogos_resultados))  
                else:
                    linha_completa.append('')  

               
                if linha_completa[0] != "Classificação":  
                    tabela_completa.append(linha_completa)


print(tabulate(tabela_completa, headers='firstrow', tablefmt='grid'))


+----+---------------+--------+-----+-----+-----+-----+-----+------+------+------+-----+--------------+
|    | Posição       |   Time |   P |   J |   V |   E |   D |   GP |   GC |   SG |   % | ÚLT. JOGOS   |
+====+===============+========+=====+=====+=====+=====+=====+======+======+======+=====+==============+
|  1 | Botafogo      |      0 |  60 |  29 |  18 |   6 |   5 |   47 |   25 |   22 |  68 | V V V E V    |
+----+---------------+--------+-----+-----+-----+-----+-----+------+------+------+-----+--------------+
|  2 | Palmeiras     |      0 |  57 |  29 |  17 |   6 |   6 |   46 |   20 |   26 |  65 | V V V V E    |
+----+---------------+--------+-----+-----+-----+-----+-----+------+------+------+-----+--------------+
|  3 | Fortaleza     |      0 |  55 |  29 |  16 |   7 |   6 |   38 |   29 |    9 |  63 | D E V V D    |
+----+---------------+--------+-----+-----+-----+-----+-----+------+------+------+-----+--------------+
|  4 | Flamengo      |      0 |  51 |  28 |  15 |   6 |   7 |   